# SRA Testing Module
For scratch testing of the SRA module.

In [1]:
# Enable autorealod so that modified files can be re-loaded
# with out restarting the kernel.
%load_ext autoreload
%autoreload 2

In [2]:
%pwd

'/home/tylerbiggs/git/pynome/tests'

In [3]:
# Add the path of the pynome project.
import os
import csv
import sys
sys.path.insert(0, os.path.abspath('..'))

In [53]:
from pynome.sra import (
    build_sra_query_string,
    run_sra_query,
    fetch_sra_info,
    parse_sra_query_response,
    get_SRA_accession,
)

In [54]:
# print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['EXPERIMENT']['@accession'])
# print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['SUBMISSION']['@accession'])
# print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['STUDY']['@accession'])
# print(fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['SAMPLE']['@accession'])

In [84]:
def download_sra_json(taxonomy_id_list, base_download_path=None):
    """
    Downloads the SRA metadata for each ID found in the
    `taxonomy_id_list`. These files are saved under a series of
    two-digit file  paths generated from the SRA accession number.

    :param taxonomy_id_list:
        A list of taxonomy identification values.
    :param base_download_path:
        The base location where the SRA accession number folders
        will be placed.
    :return:
        A list of success or failures, indexed the same as the
        input `taxonomy_id_list`.
    """

    # Create the output status dictionary to track whether a given
    # taxonomy ID was downloaded successfully or not.
    status_list = dict()

    # For each of the taxonomy ID numbers provided.
    for id in taxonomy_id_list:

        # Generate the corresponding query.
        query = build_sra_query_string(id)
        print("\nQUERY: \n", query)

        # Run the query.
        query_response = run_sra_query(query)
        print("\nRESPONSE: \n", query_response)

        # Parse the response, get the list of SRA identification
        # numbers so that the corresponding metadata can be
        # downloaded.
        fetch_id_list = parse_sra_query_response(query_response)
        print("\nFETCHED ID LIST: \n", fetch_id_list)
        
        # If there are any accession values found.
        if fetch_id_list is not None:

            # Iterate through the fetch ID numbers.
            for fetch_id in fetch_id_list:

                # Get the desired *.json file associated with
                # the current fetch ID.
                print('FETCH ID', fetch_id)
                fetch_result = fetch_sra_info(fetch_id)
                print("\nFETCH RESULT:\n", fetch_result)

                # Get the ERR or SRR from the fetched result.
                SRA_accession = get_SRA_accession(fetch_result)
                print("\nSRA accession:\n", SRA_accession)
            
                for sra_id in SRA_accession_list:
                    

In [56]:
tax_id_list = [1290391,999810,1124627,1356009,1094988,1094985,1094992,1094990,1094989,294748]

In [57]:
test_fetch_result = fetch_sra_info('149157')

In [60]:
test_fetch_result2 = fetch_sra_info('147979')

In [71]:
test_fetch_result2['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['RUN_SET']['RUN'][0]['@accession']

'SRR489777'

In [58]:
test_fetch_result['EXPERIMENT_PACKAGE_SET']['EXPERIMENT_PACKAGE']['RUN_SET']['RUN']['@accession']

'SRR447617'

In [85]:
download_sra_json(['1094985'])


QUERY: 
 txid1094985[Organism:noexp]+AND+biomol+rna[Properties]+AND+platform+illumina[Properties]+AND+100:1000[ReadLength]+AND+"paired"[Layout]

RESPONSE: 
 OrderedDict([('eSearchResult', OrderedDict([('Count', '1'), ('RetMax', '1'), ('RetStart', '0'), ('IdList', OrderedDict([('Id', '244092')])), ('TranslationSet', None), ('TranslationStack', OrderedDict([('TermSet', [OrderedDict([('Term', 'txid1094985[Organism:noexp]'), ('Field', 'Organism'), ('Count', '3'), ('Explode', 'N')]), OrderedDict([('Term', 'biomol rna[Properties]'), ('Field', 'Properties'), ('Count', '779724'), ('Explode', 'N')]), OrderedDict([('Term', 'platform illumina[Properties]'), ('Field', 'Properties'), ('Count', '3177818'), ('Explode', 'N')]), OrderedDict([('Term', '00000000100[ReadLength]'), ('Field', 'ReadLength'), ('Count', '0'), ('Explode', 'N')]), OrderedDict([('Term', '00000001000[ReadLength]'), ('Field', 'ReadLength'), ('Count', '0'), ('Explode', 'N')]), OrderedDict([('Term', '"paired"[Layout]'), ('Field', 'L

In [77]:
download_sra_json(tax_id_list)


QUERY: 
 txid1290391[Organism:noexp]+AND+biomol+rna[Properties]+AND+platform+illumina[Properties]+AND+100:1000[ReadLength]+AND+"paired"[Layout]

RESPONSE: 
 OrderedDict([('eSearchResult', OrderedDict([('Count', '0'), ('RetMax', '0'), ('RetStart', '0'), ('IdList', None), ('TranslationSet', None), ('TranslationStack', OrderedDict([('TermSet', [OrderedDict([('Term', 'txid1290391[Organism:noexp]'), ('Field', 'Organism'), ('Count', '1'), ('Explode', 'N')]), OrderedDict([('Term', 'biomol rna[Properties]'), ('Field', 'Properties'), ('Count', '779724'), ('Explode', 'N')]), OrderedDict([('Term', 'platform illumina[Properties]'), ('Field', 'Properties'), ('Count', '3177818'), ('Explode', 'N')]), OrderedDict([('Term', '00000000100[ReadLength]'), ('Field', 'ReadLength'), ('Count', '0'), ('Explode', 'N')]), OrderedDict([('Term', '00000001000[ReadLength]'), ('Field', 'ReadLength'), ('Count', '0'), ('Explode', 'N')]), OrderedDict([('Term', '"paired"[Layout]'), ('Field', 'Layout'), ('Count', '2367418

HTTPError: HTTP Error 400: Bad Request